In [1]:
import numpy as np
import torch
import torch.optim as optim
import torch.nn as nn
import random
from collections import OrderedDict

torch.set_printoptions(edgeitems=2)

In [2]:
#折射率
def n(omega):
    if(omega>1e14):
        t=100
        a1=4.582
        a2=0.09921
        a3=0.2109
        a4=-0.02194
        b1=0.00000022971
        b2=0.000000052716
        b3=-0.000000049143
        c=88506.25
        t1=t+273.15
        
        w=(2*np.pi*3e8/omega)*1e6
        c1=w*w-(a3+b3*(t1*t1-c))*(a3+b3*(t1*t1-c))
        c2=a1+b1*(t1*t1-c)+(a2+b2*(t1*t1-c))/c1+a4*w*w
        y=np.sqrt(abs(c2))
    else:
        d1=19.9
        d2=44
        d3=4.533e12*2*np.pi
        d4=0.426e12*2*np.pi
        d5=2.9176e13
        
        e=d1+(d2-d1)*d5**2/(d3**2+1.0j*omega*d4-omega**2)
        y=np.real(np.sqrt(abs(e)))

    return y

In [3]:
def kk(f):
    z=f*n(f)/3e8
    return z

In [4]:
def funct(val):
    G=np.zeros([2,300])
    d=np.zeros([2,100])
        
    d[1,:] = val
        
    for j in range(100):
        d[0,j] = j*3e-6
        
    for k in range(300):
        f_thz=0.1e12+k*0.15e12
        f_nir=3e8/800e-9
        G[0,k]=-kk(2*np.pi*f_nir)+kk(2*np.pi*(f_nir-f_thz))+kk(2*np.pi*f_thz)
        
    for ii in range(300):
        for jj in range(100):
            if jj == 99:
                break
            G[1,ii]=G[1,ii]+d[1,jj]*np.exp(1.0j*G[0,ii]*d[0,jj+1])-np.exp(1.0j*G[0,ii]*d[0,jj])/(1.0j*G[0,ii])
        
    peak = np.max(np.abs(G[1,:]))
    G_au = np.abs(G[1,:])/peak
        
    max_index = G_au.tolist().index(1)
        
    lo = max_index 
    # 前向搜索
    while G_au[lo] > 0.5 and lo > 0:
        lo -= 1
        if G_au[lo] <= 0.5:
            break 

    hi = max_index
    # 后向搜索
    while G_au[hi] > 0.5 and hi < 299:
        hi += 1
        if G_au[hi] <= 0.5:
            break

    if(hi==299 and G_au[hi]>0.5)or(lo==0 and G_au[lo]>0.5):
        bw=0.01
    else:
        bw = hi - lo
    return bw

In [5]:
def map(val):
    for i in range(len(val)):
        if val[i] > 0:
            val[i] = 1
        else:
            val[i] = -1
    return val

In [6]:
def loss_fn(output):
    output = map(output)
    #val = funct(output)
    val = sum(output)
    loss = 100/val
    loss = torch.tensor(loss)
    return loss

In [7]:
def training_loop(n_epochs, optimizer, model, loss_fn):
    val = torch.ones(100)
    a = torch.ones(1)
    for epoch in range(1, n_epochs + 1):
        out = model(val).detach() # <1>
        
        # loss = loss_fn(out).requires_grad_()
        loss = loss_fn(out, a).requires_grad_()
        
        optimizer.zero_grad()
        loss.backward() # <2>
        optimizer.step()

        if epoch == 1 or epoch % 5 == 0:
            print('Epoch {}, loss {}, out{}'.format(
                epoch, float(loss), out))

In [8]:
neuron_count = 300

seq_model = nn.Sequential(OrderedDict([
    ('hidden_linear', nn.Linear(100, neuron_count)),
    ('hidden_activation', nn.ReLU()),
    ('output_linear', nn.Linear(neuron_count, 100))
]))
optimizer = optim.SGD(seq_model.parameters(), lr=1e-4)
training_loop(
    n_epochs = 50, 
    optimizer = optimizer,
    model = seq_model,
    loss_fn = nn.MSELoss())

C:\Users\20241\Anaconda3\envs\pytorch\lib\site-packages\torch\nn\modules\loss.py:431: UserWarning: Using a target size (torch.Size([1])) that is different to the input size (torch.Size([100])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


Epoch 1, loss 1.0370107889175415, outtensor([ 0.6905,  0.5763,  0.0996,  0.6856, -0.1091,  0.4390,  0.1200,  0.3836,
         0.1861, -0.4535, -0.4741,  0.1909, -0.2311, -0.2071, -0.0600, -0.0746,
        -0.0453, -0.1659, -0.0792, -0.3701,  0.2886,  0.2971, -0.3857, -0.3353,
        -0.1416, -0.0106,  0.1773, -0.2064, -0.1346, -0.1499,  0.0760,  0.0178,
        -0.2092,  0.2804,  0.3184,  0.0308,  0.0722,  0.1011,  0.1467,  0.0185,
        -0.1329, -0.1916, -0.2707,  0.5302, -0.1857,  0.3334, -0.2594, -0.1899,
        -0.1032, -0.7277,  0.0797,  0.3476,  0.3279,  0.3559,  0.1719, -0.2431,
         0.0101,  0.5859, -0.1115, -0.1187,  0.0494, -0.0873, -0.0011,  0.1081,
         0.0179,  0.6665, -0.0607,  0.4690,  0.2567, -0.1523,  0.2430,  0.1316,
         0.1810, -0.1867, -0.1952, -0.0208,  0.5185, -0.0325, -0.0808, -0.2208,
        -0.3256,  0.2664, -0.1071, -0.0230, -0.0628, -0.1156,  0.3619, -0.2283,
         0.3857,  0.0556, -0.0842,  0.0727, -0.2320, -0.8158, -0.0820, -0.0929,
   